In [1]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
@tool
def multiply(a: int, b: int) -> int:
    """Given two numbers a and b, return their product."""
    return a * b

In [3]:
multiply.invoke({"a": 2, "b": 3})

6

In [4]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [47]:
from langchain_ollama import ChatOllama

chat = ChatOllama(
    model="llama3.2:latest",
    temperature=.5
)

In [48]:
llm_with_tools=chat.bind_tools([multiply])

In [51]:
llm_with_tools.invoke("Can you multiply 2 and 3?")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-08-09T16:32:15.3852056Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1513029400, 'load_duration': 56980000, 'prompt_eval_count': 175, 'prompt_eval_duration': 66530700, 'eval_count': 22, 'eval_duration': 1388455200, 'model_name': 'llama3.2:latest'}, id='run--038ff91e-9aa0-4cd2-ab74-ddcc070b2564-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': '5367109b-b895-4abd-9028-65b508d24ac8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 175, 'output_tokens': 22, 'total_tokens': 197})

In [52]:
llm_with_tools.invoke("Can you multiply 2 and 3?").tool_calls[0]

{'name': 'multiply',
 'args': {'a': '2', 'b': '3'},
 'id': 'b63527b6-7989-421a-a28d-853d5602df68',
 'type': 'tool_call'}

In [53]:
multiply.invoke({'name': 'multiply',
 'args': {'a': '2', 'b': '3'},
 'id': 'b63527b6-7989-421a-a28d-853d5602df68',
 'type': 'tool_call'})

ToolMessage(content='6', name='multiply', tool_call_id='b63527b6-7989-421a-a28d-853d5602df68')